# Objective
## 1. Pre-process the data
## 1. Fit a LDA model to the cleaned and pre-processed data
## 2. Group topics together based on their Hellinger distance
## 3. Label comments with topics


# Import the data and the modules

In [ ]:
!pip install gensim==3.8.3
!pip install pyLDAvis==3.2.0
!pip install wordcloud==1.8.1

/shared-libs/python3.7/py-core/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
You should consider upgrading via the '/root/venv/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/root/venv/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/root/venv/bin/python -m pip install --upgrade pip' command.


In [ ]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore")

import pandas as pd
import matplotlib.pyplot as plt
import tqdm
import gensim
import pyLDAvis.gensim
import pickle 
import pyLDAvis
import pyLDAvis.gensim
import nltk
import numpy as np
np.random.seed(2018)

from gensim.models import CoherenceModel
from gensim.utils import simple_preprocess
from gensim.matutils import hellinger
from gensim.parsing.preprocessing import STOPWORDS
from gensim.parsing.preprocessing import remove_stopwords
from gensim import corpora, models
from pprint import pprint
from nltk.stem import WordNetLemmatizer, SnowballStemmer 
from nltk.stem.porter import *
from wordcloud import WordCloud
from textblob import TextBlob
from sklearn.model_selection import train_test_split

nltk.download('wordnet')

all_stopwords_gensim = STOPWORDS.union(set(['awww', 
                                            'afternoon', 
                                            'mmmm', 
                                            'haha', 
                                            'amazing',
                                            'awesome',
                                            'gorgeous',
                                            'sexy',
                                            'fantastic',
                                            'beautiful',
                                            'perfect',
                                            'favorite',
                                            'great',
                                            'good',
                                            'sweet',
                                            'best',
                                            'super',
                                            'nice',
                                            'xoxo',
                                            'crazy',
                                            'soooo',
                                            'incredible',
                                            'absolute',
                                            'delicioous',
                                            'mmmmm',
                                            'gonna',
                                            'bruh',
                                            'yeah',
                                            'perfect',
                                            'follow',
                                            'sooo',
                                            'yummi',
                                            'hello',
                                            'woooow',
                                            'thank',
                                            "thanks",
                                            "get",
                                            "superb",
                                            "aboslute",
                                            "absolutely",
                                            "cool",
                                            "beautiful",
                                            "maybe",
                                            "damn",
                                            "that",
                                            "perfect",
                                            "shes",
                                            "yummy",
                                            "yummi",
                                            "lmao",
                                            "yooou",
                                            "magnifique",
                                            "love",
                                            "lovely",
                                            "mmmmmmm",
                                            "yoou",
                                            "wooow",
                                            "xoxoxo",
                                            "sooooo",
                                            "wowww",
                                            "yaay",
                                            "ohhh",
                                            "hotttt",
                                            "whoa",
                                            "hehe",
                                            "dang",
                                            "xxxx",
                                            "goddamn",
                                            "sublime",
                                            "fabulous",
                                            "fuckkk",
                                            "gooood",
                                            "xoxo",
                                            "soooooo",
                                            "mmmmmm",
                                            "hahaha",
                                            "xxxxx",
                                            "nicee",
                                            "veeery",
                                            "sexxxxyy",
                                            "delici",
                                            "sensual",
                                            "greatest",                                            
                                            "didnt"
                                            "will",
                                            "cute",
                                            "kind",
                                            "couldn",
                                            "magnific",
                                            "gostosa",
                                            "deliciosa",
                                            "delicia",
                                            "wonderful",
                                            "genuin",
                                            "love",
                                            "instead",
                                            "huge",
                                            "impress",
                                            "delicioso",
                                            "wish",
                                            "hotter",
                                            "begin",
                                            "unbeliev",
                                            "congratul",
                                            "esta",
                                            "como",
                                            "boca",
                                            "buen",
                                            "est",
                                            "quiero",
                                            "youu",
                                            "doesnt",
                                            "beauti",
                                            "bravo",
                                            "brilliant",
                                            "prettiest",
                                            "believ",
                                            "wanna",
                                            "eat",
                                            "astonish",
                                            "yesss",
                                            "worst",
                                            "princess",
                                            "bella",
                                            "aussi",
                                            "superb",
                                            "woow",
                                            "hilari",
                                            "divin",
                                            "yessss",
                                            "have",
                                            "goddess",
                                            "hmmm",
                                            "hermosa",
                                            "para",
                                            "pero",
                                            "hermoso",
                                            "insan",
                                            "later",
                                            "sexiest",
                                            "wasn",
                                            "woah",
                                            "oooh",
                                            "sehr",
                                            "geil",
                                            "damm",
                                            "flawless",
                                            "yall"]))


stemmer = SnowballStemmer("english")

/shared-libs/python3.7/py-core/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
data = pd.read_csv("ReadyForAnalysis.csv")
cols = data.columns

# Add all of the other columns
data_full = pd.DataFrame(data.loc[(data.clean==True) & (data.is_eng==True), ["title", 
                                                                             "views", 
                                                                             "up_votes", 
                                                                             "down_votes", 
                                                                             "percent", 
                                                                             "author",
                                                                             "author_subscriber",
                                                                             "categories",
                                                                             "tags",
                                                                             "production",
                                                                             "description",
                                                                             "duration",
                                                                             "upload_date",
                                                                             "pornstars",
                                                                             "download_urls",
                                                                             "thumbnail_url",
                                                                             "number_of_comment",
                                                                             "url",
                                                                             "error",
                                                                             "repeat_n_times",
                                                                             "comms",
                                                                             "clean",
                                                                             "is_eng"
                                                                            ]])

data_text = data_full = pd.DataFrame(data.loc[(data.clean==True) & (data.is_eng==True), ["comms"]])

data_text = data_text.dropna(subset=['comms'])
data_text = data_text.reset_index(drop=True)
data_text['index'] = data_text.index

documents = data_text

# PT1: Data pre-processing
    ## -Words are lemmatized — words in third person are changed to first person and verbs in past and future tenses are changed into present.
    ## -Words are stemmed — words are reduced to their root form. (Happiness --> Happy)
    ## -Tokenization: Split the text into sentences and the sentences into words. Lowercase the words and remove punctuation.
    ## -Words that have fewer than 3 characters are removed.
    ## -All stopwords are removed.

## Lemmatize and stemming function

In [ ]:
def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))

def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in all_stopwords_gensim and len(token) > 3:
            result.append(lemmatize_stemming(token))
    return result

## Run the function over the whole dataset

In [ ]:
processed_docs = documents["comms"].map(preprocess)

0    [horni, anymor, want, happi]
1                 [silenc, wench]
2                          [feel]
3                              []
4                    [want, love]
Name: comms, dtype: object

## Also store the results in the full dataframe

In [ ]:
processed_coms = processed_docs
data_full["processed_coms"] = processed_coms

## Remove empty comments

In [ ]:
print(len(data_full))

#Remove empty rows
data_full = data_full.dropna(subset=['processed_coms'])
data_full = data_full.reset_index(drop=True)

print(len(data_full))

#Remove empty comments
data_full = data_full[data_full['processed_coms'].map(lambda d: len(d)) > 0]
data_full = data_full.reset_index(drop=True)

print(len(data_full))

#Assign the new values to processed_docs
processed_docs = data_full["processed_coms"]

201449
120876
99087


## Bigrams

Forming bigrams and trigram from documents (comments) is **important in topic modelling** because certain words tend to **occur together**. For example, "french" refers to the *language*, "revolution" to the *astronomical phenomenon*, but "french" + "revolution" refers to the *historical event*.

In [ ]:
# Build the bigram models
bigram = gensim.models.phrases.Phrases(processed_docs, min_count=3, threshold=10)
bigram_mod = gensim.models.phrases.Phraser(bigram)

['horni_anymor', 'want', 'happi']


In [ ]:
processed_docs_bigram = {"bigram": []}
for i in range(len(processed_docs)) :
    big = bigram[processed_docs[i]]
    
    processed_docs_bigram["bigram"].append(big)

processed_docs_bigram = pd.DataFrame(processed_docs_bigram)

## Visualisation of the whole processed document

In [ ]:
processed_docs = processed_docs_bigram["bigram"]
data_full["processed_coms"] = processed_docs

# Save the processed data

In [ ]:
data_full.to_csv(r'Processed_Docs_full.csv')

# PT2: Training the model

## We create a custom dictionary based on our dataset

In [ ]:
dictionary = gensim.corpora.Dictionary(processed_docs)

0 happi
1 horni_anymor
2 wanna
3 want
4 silenc
5 wench
6 feel
7 love
8 cook
9 know
10 proper


### We filter out words that:
    #### -Are present in less than 15 comments (too rare)
    #### -In more than 50% of the comments (too numerous --> stopwords)
    #### - After the first two steps, we only keep the first 100k tokens/words (further filtering of stopwords)

In [ ]:
dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)

In [ ]:
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]

[(5, 1)]

## Model evaluation and tuning (sensitivity analysis): coherence = C_v

In [ ]:
#Compute coherence score for range of topics
Sensitivity_analysis = {'Topics': [],
                 'Alpha': [],
                 'Beta': [],
                 'Coherence': [],
                        'Perplexity': []
                }

#Topics(K):
topics_range = range(3,66,1)

#Alpha:
alpha = list(np.arange(0.01, 1, 0.3))

#Beta:
beta = list(np.arange(0.01, 1, 0.3))

#Number of passes:
num_passes = 2

#Create training and holdout data
train, test = train_test_split(processed_docs, test_size=0.2)

# Training dictionary and BoW
dictionary = gensim.corpora.Dictionary(train)
dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)

bow_corpus = [dictionary.doc2bow(doc) for doc in train]

# tfidf = models.TfidfModel(bow_corpus)
# corpus_tfidf = tfidf[bow_corpus]


In [ ]:
#Loop over the range of values
for k in topics_range:
    print(k)
    print("topics out of")
    print(max(topics_range))
    print("-------------------------------------")
    for a in alpha:
        for b in beta:
            
            lda_model = gensim.models.LdaMulticore(corpus=bow_corpus,
                                           id2word=dictionary,
                                           num_topics=k, 
                                           random_state=100,
                                           chunksize=100,
                                           passes=num_passes,
                                           alpha=a,
                                           eta=b)
            coherence_model_lda = CoherenceModel(model=lda_model, 
                                                       texts=test, 
                                                       dictionary=dictionary, 
                                                       coherence='c_v')
            coherence_lda = coherence_model_lda.get_coherence()
            
            perplexity_lda = lda_model.log_perplexity(bow_corpus)
            
            Sensitivity_analysis["Topics"].append(k)
            Sensitivity_analysis["Alpha"].append(a)
            Sensitivity_analysis["Beta"].append(b)
            Sensitivity_analysis["Coherence"].append(coherence_lda)
            Sensitivity_analysis["Perplexity"].append(perplexity_lda)
            

3
topics out of
65
-------------------------------------
4
topics out of
65
-------------------------------------
5
topics out of
65
-------------------------------------
6
topics out of
65
-------------------------------------
7
topics out of
65
-------------------------------------
8
topics out of
65
-------------------------------------
9
topics out of
65
-------------------------------------
10
topics out of
65
-------------------------------------
11
topics out of
65
-------------------------------------
12
topics out of
65
-------------------------------------
13
topics out of
65
-------------------------------------
14
topics out of
65
-------------------------------------
15
topics out of
65
-------------------------------------
16
topics out of
65
-------------------------------------
17
topics out of
65
-------------------------------------
18
topics out of
65
-------------------------------------
19
topics out of
65
-------------------------------------
20
topics out of
65
-

28
topics out of
65
-------------------------------------
29
topics out of
65
-------------------------------------
30
topics out of
65
-------------------------------------
31
topics out of
65
-------------------------------------
32
topics out of
65
-------------------------------------
33
topics out of
65
-------------------------------------
34
topics out of
65
-------------------------------------
35
topics out of
65
-------------------------------------
36
topics out of
65
-------------------------------------


37
topics out of
65
-------------------------------------
38
topics out of
65
-------------------------------------


39
topics out of
65
-------------------------------------
40
topics out of
65
-------------------------------------
41
topics out of
65
-------------------------------------
42
topics out of
65
-------------------------------------
43
topics out of
65
-------------------------------------
44
topics out of
65
-------------------------------------
45
topics out of
65
-------------------------------------
46
topics out of
65
-------------------------------------
47
topics out of
65
-------------------------------------
48
topics out of
65
-------------------------------------
49
topics out of
65
-------------------------------------
50
topics out of
65
-------------------------------------
51
topics out of
65
-------------------------------------
52
topics out of
65
-------------------------------------
53
topics out of
65
-------------------------------------
54
topics out of
65
-------------------------------------
55
topics out of
65
-------------------------------------
56
topics out 

In [ ]:
#Save the results internally
Sensitivity_analysis_pd = pd.DataFrame.from_dict(Sensitivity_analysis)

#Save the results externally
Sensitivity_analysis_pd.to_csv('Sensitivity_analysis.csv', index=False)

## Additional loop for alpha symmetric

In [ ]:
#Compute coherence score for range of topics
Sensitivity_analysis = {'Topics': [],
                 'Alpha': [],
                 'Beta': [],
                 'Coherence': [],
                        'Perplexity': []
                }

#Topics(K):
topics_range = range(3,66,1)

#Alpha:
alpha = "symmetric"

#Beta:
beta = list(np.arange(0.01, 1, 0.3))

#Number of passes:
num_passes = 2

#Create training and holdout data
train, test = train_test_split(processed_docs, test_size=0.2)

# Training dictionary and BoW
dictionary = gensim.corpora.Dictionary(train)
dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)

bow_corpus = [dictionary.doc2bow(doc) for doc in train]

# tfidf = models.TfidfModel(bow_corpus)
# corpus_tfidf = tfidf[bow_corpus]


#Loop over the range of values
for k in topics_range:
    print(k)
    print("topics out of")
    print(max(topics_range))
    print("-------------------------------------")
    
    for b in beta:
            
        lda_model = gensim.models.LdaMulticore(corpus=bow_corpus,
                                           id2word=dictionary,
                                           num_topics=k, 
                                           random_state=100,
                                           chunksize=100,
                                           passes=num_passes,
                                           alpha=alpha,
                                           eta=b)
        coherence_model_lda = CoherenceModel(model=lda_model, 
                                                       texts=test, 
                                                       dictionary=dictionary, 
                                                       coherence='c_v')
        coherence_lda = coherence_model_lda.get_coherence()
            
        perplexity_lda = lda_model.log_perplexity(bow_corpus)
            
        Sensitivity_analysis["Topics"].append(k)
        Sensitivity_analysis["Alpha"].append(alpha)
        Sensitivity_analysis["Beta"].append(b)
        Sensitivity_analysis["Coherence"].append(coherence_lda)
        Sensitivity_analysis["Perplexity"].append(perplexity_lda)
            
            

/shared-libs/python3.7/py-core/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
3
topics out of
65
-------------------------------------
4
topics out of
65
-------------------------------------
5
topics out of
65
-------------------------------------
6
topics out of
65
-------------------------------------


7
topics out of
65
-------------------------------------
8
topics out of
65
-------------------------------------
9
topics out of
65
-------------------------------------
10
topics out of
65
-------------------------------------
11
topics out of
65
-------------------------------------
12
topics out of
65
-------------------------------------
13
topics out of
65
-------------------------------------
14
topics out of
65
-------------------------------------
15
topics out of
65
-------------------------------------
16
topics out of
65
-------------------------------------
17
topics out of
65
-------------------------------------
18
topics out of
65
-------------------------------------
19
topics out of
65
-------------------------------------
20
topics out of
65
-------------------------------------
21
topics out of
65
-------------------------------------
22
topics out of
65
-------------------------------------
23
topics out of
65
-------------------------------------
24
topics out of


In [ ]:
#Save the results internally
Sensitivity_analysis_pd = pd.DataFrame.from_dict(Sensitivity_analysis)

#Save the results externally
Sensitivity_analysis_pd.to_csv('Sensitivity_analysis_ALPHAsymmetric_3-65.csv', index=False)

/shared-libs/python3.7/py-core/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


/shared-libs/python3.7/py-core/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


## Alpha asymmetric

In [ ]:
#Compute coherence score for range of topics
Sensitivity_analysis = {'Topics': [],
                 'Alpha': [],
                 'Beta': [],
                 'Coherence': [],
                        'Perplexity': []
                }

#Topics(K):
topics_range = range(3,66,1)

#Alpha:
alpha = "asymmetric"

#Beta:
beta = list(np.arange(0.01, 1, 0.3))

#Number of passes:
num_passes = 2

#Create training and holdout data
train, test = train_test_split(processed_docs, test_size=0.2)

# Training dictionary and BoW
dictionary = gensim.corpora.Dictionary(train)
dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)

bow_corpus = [dictionary.doc2bow(doc) for doc in train]

# tfidf = models.TfidfModel(bow_corpus)
# corpus_tfidf = tfidf[bow_corpus]


#Loop over the range of values
for k in topics_range:
    print(k)
    print("topics out of")
    print(max(topics_range))
    print("-------------------------------------")
    
    for b in beta:
            
        lda_model = gensim.models.LdaMulticore(corpus=bow_corpus,
                                           id2word=dictionary,
                                           num_topics=k, 
                                           random_state=100,
                                           chunksize=100,
                                           passes=num_passes,
                                           alpha=alpha,
                                           eta=b)
        coherence_model_lda = CoherenceModel(model=lda_model, 
                                                       texts=test, 
                                                       dictionary=dictionary, 
                                                       coherence='c_v')
        coherence_lda = coherence_model_lda.get_coherence()
            
        perplexity_lda = lda_model.log_perplexity(bow_corpus)
            
        Sensitivity_analysis["Topics"].append(k)
        Sensitivity_analysis["Alpha"].append(alpha)
        Sensitivity_analysis["Beta"].append(b)
        Sensitivity_analysis["Coherence"].append(coherence_lda)
        Sensitivity_analysis["Perplexity"].append(perplexity_lda)
            

/shared-libs/python3.7/py-core/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
3
topics out of
65
-------------------------------------
4
topics out of
65
-------------------------------------
5
topics out of
65
-------------------------------------
6
topics out of
65
-------------------------------------
7
topics out of
65
-------------------------------------
8
topics out of
65
-------------------------------------
9
topics out of
65
-------------------------------------
10
topics out of
65
-------------------------------------
11
topics out of
65
-------------------------------------
12
topics out of
65
-------------------------------------
13
topics out of
65
---------------------------------

/shared-libs/python3.7/py-core/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
3
topics out of
65
-------------------------------------
4
topics out of
65
-------------------------------------
5
topics out of
65
-------------------------------------
6
topics out of
65
-------------------------------------
7
topics out of
65
-------------------------------------
8
topics out of
65
-------------------------------------
9
topics out of
65
-------------------------------------
10
topics out of
65
-------------------------------------
11
topics out of
65
-------------------------------------
12
topics out of
65
-------------------------------------


13
topics out of
65
-------------------------------------
14
topics out of
65
-------------------------------------
15
topics out of
65
-------------------------------------
16
topics out of
65
-------------------------------------
17
topics out of
65
-------------------------------------
18
topics out of
65
-------------------------------------
19
topics out of
65
-------------------------------------
20
topics out of
65
-------------------------------------
21
topics out of
65
-------------------------------------
22
topics out of
65
-------------------------------------
23
topics out of
65
-------------------------------------
24
topics out of
65
-------------------------------------
25
topics out of
65
-------------------------------------
26
topics out of
65
-------------------------------------
27
topics out of
65
-------------------------------------
28
topics out of
65
-------------------------------------
29
topics out of
65
-------------------------------------
30
topics out 

In [ ]:
#Save the results internally
Sensitivity_analysis_pd = pd.DataFrame.from_dict(Sensitivity_analysis)

#Save the results externally
Sensitivity_analysis_pd.to_csv('Sensitivity_analysis_ALPHAasymmetric_3-65.csv', index=False)

/shared-libs/python3.7/py-core/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


/shared-libs/python3.7/py-core/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


### Alpha and Beta symmetric

In [ ]:
#Compute coherence score for range of topics
Sensitivity_analysis = {'Topics': [],
                 'Alpha': [],
                 'Beta': [],
                 'Coherence': [],
                        'Perplexity': []
                }

#Topics(K):
topics_range = range(3,66,1)

#Alpha:
alpha = "symmetric"

#Beta:
beta = "symmetric"

#Number of passes:
num_passes = 2

#Create training and holdout data
train, test = train_test_split(processed_docs, test_size=0.2)

# Training dictionary and BoW
dictionary = gensim.corpora.Dictionary(train)
dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)

bow_corpus = [dictionary.doc2bow(doc) for doc in train]

# tfidf = models.TfidfModel(bow_corpus)
# corpus_tfidf = tfidf[bow_corpus]


#Loop over the range of values
for k in topics_range:
    print(k)
    print("topics out of")
    print(max(topics_range))
    print("-------------------------------------")
            
    lda_model = gensim.models.LdaMulticore(corpus=bow_corpus,
                                           id2word=dictionary,
                                           num_topics=k, 
                                           random_state=100,
                                           chunksize=100,
                                           passes=num_passes,
                                           alpha=alpha,
                                           eta=beta)
    coherence_model_lda = CoherenceModel(model=lda_model, 
                                                       texts=test, 
                                                       dictionary=dictionary, 
                                                       coherence='c_v')
    coherence_lda = coherence_model_lda.get_coherence()
            
    perplexity_lda = lda_model.log_perplexity(bow_corpus)
            
    Sensitivity_analysis["Topics"].append(k)
    Sensitivity_analysis["Alpha"].append(alpha)
    Sensitivity_analysis["Beta"].append(beta)
    Sensitivity_analysis["Coherence"].append(coherence_lda)
    Sensitivity_analysis["Perplexity"].append(perplexity_lda)
            

/shared-libs/python3.7/py-core/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
3
topics out of
65
-------------------------------------
4
topics out of
65
-------------------------------------
5
topics out of
65
-------------------------------------
6
topics out of
65
-------------------------------------
7
topics out of
65
-------------------------------------
8
topics out of
65
-------------------------------------
9
topics out of
65
-------------------------------------
10
topics out of
65
-------------------------------------
11
topics out of
65
-------------------------------------
12
topics out of
65
-------------------------------------
13
topics out of
65
---------------------------------

/shared-libs/python3.7/py-core/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
3
topics out of
65
-------------------------------------
4
topics out of
65
-------------------------------------
5
topics out of
65
-------------------------------------
6
topics out of
65
-------------------------------------
7
topics out of
65
-------------------------------------
8
topics out of
65
-------------------------------------
9
topics out of
65
-------------------------------------
10
topics out of
65
-------------------------------------
11
topics out of
65
-------------------------------------
12
topics out of
65
-------------------------------------
13
topics out of
65
---------------------------------

In [ ]:
#Save the results internally
Sensitivity_analysis_pd = pd.DataFrame.from_dict(Sensitivity_analysis)

#Save the results externally
Sensitivity_analysis_pd.to_csv('Sensitivity_analysis_symmetric_3-65.csv', index=False)

/shared-libs/python3.7/py-core/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


/shared-libs/python3.7/py-core/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


### Beta symmetric

In [ ]:
#Compute coherence score for range of topics
Sensitivity_analysis = {'Topics': [],
                 'Alpha': [],
                 'Beta': [],
                 'Coherence': [],
                        'Perplexity': []
                }

#Topics(K):
topics_range = range(3,66,1)

#Alpha:
alpha = list(np.arange(0.01, 1, 0.3))

#Beta:
beta = "symmetric"

#Number of passes:
num_passes = 2

#Create training and holdout data
train, test = train_test_split(processed_docs, test_size=0.2)

# Training dictionary and BoW
dictionary = gensim.corpora.Dictionary(train)
dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)

bow_corpus = [dictionary.doc2bow(doc) for doc in train]

# tfidf = models.TfidfModel(bow_corpus)
# corpus_tfidf = tfidf[bow_corpus]


#Loop over the range of values
for k in topics_range:
    print(k)
    print("topics out of")
    print(max(topics_range))
    print("-------------------------------------")
    for a in alpha:
            
        lda_model = gensim.models.LdaMulticore(corpus=bow_corpus,
                                           id2word=dictionary,
                                           num_topics=k, 
                                           random_state=100,
                                           chunksize=100,
                                           passes=num_passes,
                                           alpha=a,
                                           eta=beta)
        coherence_model_lda = CoherenceModel(model=lda_model, 
                                                       texts=test, 
                                                       dictionary=dictionary, 
                                                       coherence='c_v')
        coherence_lda = coherence_model_lda.get_coherence()
            
        perplexity_lda = lda_model.log_perplexity(bow_corpus)
            
        Sensitivity_analysis["Topics"].append(k)
        Sensitivity_analysis["Alpha"].append(a)
        Sensitivity_analysis["Beta"].append(beta)
        Sensitivity_analysis["Coherence"].append(coherence_lda)
        Sensitivity_analysis["Perplexity"].append(perplexity_lda)
            
            

/shared-libs/python3.7/py-core/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
3
topics out of
65
-------------------------------------
4
topics out of
65
-------------------------------------
5
topics out of
65
-------------------------------------
6
topics out of
65
-------------------------------------
7
topics out of
65
-------------------------------------
8
topics out of
65
-------------------------------------
9
topics out of
65
-------------------------------------
10
topics out of
65
-------------------------------------
11
topics out of
65
-------------------------------------
12
topics out of
65
-------------------------------------
13
topics out of
65
---------------------------------

/shared-libs/python3.7/py-core/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
3
topics out of
65
-------------------------------------
4
topics out of
65
-------------------------------------
5
topics out of
65
-------------------------------------
6
topics out of
65
-------------------------------------
7
topics out of
65
-------------------------------------
8
topics out of
65
-------------------------------------
9
topics out of
65
-------------------------------------
10
topics out of
65
-------------------------------------
11
topics out of
65
-------------------------------------
12
topics out of
65
-------------------------------------
13
topics out of
65
---------------------------------

48
topics out of
65
-------------------------------------
49
topics out of
65
-------------------------------------
50
topics out of
65
-------------------------------------
51
topics out of
65
-------------------------------------
52
topics out of
65
-------------------------------------
53
topics out of
65
-------------------------------------
54
topics out of
65
-------------------------------------
55
topics out of
65
-------------------------------------
56
topics out of
65
-------------------------------------
57
topics out of
65
-------------------------------------
58
topics out of
65
-------------------------------------
59
topics out of
65
-------------------------------------
60
topics out of
65
-------------------------------------
61
topics out of
65
-------------------------------------
62
topics out of
65
-------------------------------------
63
topics out of
65
-------------------------------------
64
topics out of
65
-------------------------------------
65
topics out 

In [ ]:
#Save the results internally
Sensitivity_analysis_pd = pd.DataFrame.from_dict(Sensitivity_analysis)

#Save the results externally
Sensitivity_analysis_pd.to_csv('Sensitivity_analysis_BETAsymmetric_3-65.csv', index=False)

/shared-libs/python3.7/py-core/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


/shared-libs/python3.7/py-core/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


## Generate best model based on sensitivity analysis

In [ ]:
# Dictionary and BoW
dictionary = gensim.corpora.Dictionary(processed_docs)
dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)

corpus = [dictionary.doc2bow(doc) for doc in processed_docs]

k = 57
num_passes = 10
a = 0.01
b = 0.61

lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                           id2word=dictionary,
                                           num_topics=k, 
                                           random_state=100,
                                           chunksize=100,
                                           passes=num_passes,
                                           alpha=a,
                                           eta=b)

In [ ]:
# plt.plot(Sensitivity_analysis_pd["Topics"], Sensitivity_analysis_pd["Coherence"])
# plt.xlabel('K topics')
# plt.ylabel('Coherence score (C_v)')
# plt.show()

# Save model

In [ ]:
#Save model to disk (no need to use pickle module)
lda_model.save('lda_model.model')

## To load the model

In [ ]:
# # lda_model_tfidf =  models.LdaModel.load('lda_model_tfidf.model')

# #To show topics
# lda_visualization = pyLDAvis.gensim.prepare(lda_model_tfidf, corpus_tfidf, dictionary, sort_topics=False)

# pyLDAvis.display(lda_visualization)


# PT3: Topic labelling

# Record the distance between topics

## Record the words in each topic

In [ ]:
num_topics = 57
num_words = 20

Words_in_topics = {'Topics': [],
                 'Words': [],
                 'Probability': []}

words = lda_model.show_topics(formatted=False,num_words=num_words, num_topics = num_topics)
# words

for i in range(num_topics):
    for n in range(num_words):
        word_topic = words[i][1][n][0]
        word_prob = words[i][1][n][1]

        Words_in_topics["Topics"].append(i)
        Words_in_topics["Words"].append(word_topic)
        Words_in_topics["Probability"].append(word_prob)

Words_in_topics = pd.DataFrame(Words_in_topics)

/shared-libs/python3.7/py-core/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Topics,Words,Probability
0,0,fuck,0.742537
1,0,comment,0.085933
2,0,bitch,0.023916
3,0,piec,0.004984
4,0,stupid,0.004158
...,...,...,...
1135,56,suit,0.010923
1136,56,facesit,0.009681
1137,56,unreal,0.006672
1138,56,feedback,0.005868


In [ ]:
Words_in_topics.to_csv("words_in_topics.csv")

/shared-libs/python3.7/py-core/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


## Compute the Hellinger distance

In [ ]:
#Processing function for Hellinger distance
topic_distribution = {"Topics": [],
              "Prob_distribution": []}

for i in range(num_topics):
    words_topic = Words_in_topics.loc[(Words_in_topics.Topics==i), ["Words"]].values.tolist()
    words_topic = sum(words_topic, [])
    
    prob_dist = lda_model.id2word.doc2bow(words_topic)  
    
    topic_distribution["Topics"].append(i)
    topic_distribution["Prob_distribution"].append(prob_dist)

topic_distribution = pd.DataFrame(topic_distribution)

/shared-libs/python3.7/py-core/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Topics,Prob_distribution
0,0,"[(27, 1), (40, 1), (266, 1), (659, 1), (963, 1..."
1,1,"[(37, 1), (76, 1), (78, 1), (79, 1), (270, 1),..."
2,2,"[(44, 1), (131, 1), (185, 1), (350, 1), (351, ..."
3,3,"[(348, 1), (382, 1), (383, 1), (384, 1), (456,..."
4,4,"[(21, 1), (75, 1), (319, 1), (381, 1), (387, 1..."
5,5,"[(59, 1), (123, 1), (237, 1), (241, 1), (536, ..."
6,6,"[(140, 1), (149, 1), (178, 1), (318, 1), (336,..."
7,7,"[(17, 1), (20, 1), (46, 1), (52, 1), (191, 1),..."
8,8,"[(93, 1), (141, 1), (170, 1), (200, 1), (378, ..."
9,9,"[(114, 1), (304, 1), (305, 1), (345, 1), (346,..."


In [ ]:
topic_distances = {"from": [],
                  "to": [],
                  "Hellinger_distance": []}

for i in range(num_topics):
    for y in range(num_topics):
        hellinger_distance = hellinger(sum(topic_distribution.loc[(topic_distribution.Topics == i), ["Prob_distribution"]].values.tolist(),[]),
                                      sum(topic_distribution.loc[(topic_distribution.Topics == y), ["Prob_distribution"]].values.tolist(),[]))
        
        topic_distances["from"].append(i)
        topic_distances["to"].append(y)
        topic_distances["Hellinger_distance"].append(hellinger_distance)

topic_distances = pd.DataFrame(topic_distances)

/shared-libs/python3.7/py-core/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,from,to,Hellinger_distance
0,0,0,0.000000
1,0,1,39.229294
2,0,2,41.574549
3,0,3,21.161725
4,0,4,25.185514
...,...,...,...
3244,56,52,23.280711
3245,56,53,15.708146
3246,56,54,19.687472
3247,56,55,13.372986


## Save the distance between topics

In [ ]:
topic_distances.to_csv('Topic_distances.csv', index=False)

/shared-libs/python3.7/py-core/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


## Group the topics together based on their Hellinger distance and summarise them with a label based on their word distribution

In [ ]:
Topic1 = "Duration"
Topic2 = "Looks"
Topic3 = "Cluster3"
Topic4 = "Cluster4"
Topic5 = "Cluster5"
Topic6 = "Female_attribute"
Topic7 = "Video"
Topic8 = "Skills"
Topic9 = "Intercourse"
Topic10 = "Orgasm"

# PT4: Comments labelling

### Load the data

In [ ]:
final_data = pd.read_csv("ReadyForAnalysis.csv")
cols = data.columns

# Add all of the other columns
final_data = pd.DataFrame(final_data.loc[(final_data.clean==True) & (final_data.is_eng==True), ["title", 
                                                                             "views", 
                                                                             "up_votes", 
                                                                             "down_votes", 
                                                                             "percent", 
                                                                             "author",
                                                                             "author_subscriber",
                                                                             "categories",
                                                                             "tags",
                                                                             "production",
                                                                             "description",
                                                                             "duration",
                                                                             "upload_date",
                                                                             "pornstars",
                                                                             "download_urls",
                                                                             "thumbnail_url",
                                                                             "number_of_comment",
                                                                             "url",
                                                                             "error",
                                                                             "repeat_n_times",
                                                                             "comms",
                                                                             "clean",
                                                                             "is_eng"
                                                                            ]])

final_data = final_data.dropna(subset=['comms'])
final_data = final_data.reset_index(drop=True)
final_data['index'] = final_data.index

## Label comments

### Create the labelling function

In [ ]:
def labelling(com):

    #Process the new document
    com = preprocess(com)
    
    #Compute matching topic
    topics = lda_model.show_topics(formatted=True, num_topics=57, num_words=10)
    label = pd.DataFrame([(el[0], round(el[1],2), topics[el[0]][1]) for el in lda_model[dictionary.doc2bow(com)]], 
             columns=['topic', 'weight', 'words in topic'])
    label = label.sort_values(by=['weight'], ascending = False)
    label = label.reset_index(drop=True)
    
    #Record topic and weight
    topic_n_weight = []
    
    topic = label.loc[0, "topic"]
    weight = label.loc[0, "weight"]
    
    topic_n_weight.append(topic)
    topic_n_weight.append(weight)
    
    return(topic_n_weight)

## Apply labelling function to every document in the dataset

In [ ]:
#Create columns
final_data["topic_nb"] = range(len(final_data))
# final_data["weight"] = range(len(final_data))

# Create the clusters
clust_01 = [12, 15, 11, 20, 16, 21]
clust_02 = [33, 52, 25, 30, 48, 41, 39, 50, 44, 49]
clust_03 = [43, 53, 46, 47, 54]
clust_04 = [18, 26, 37, 40]
clust_05 = [56, 55, 45, 51]
clust_06 = [14, 29, 17, 22, 19, 23, 27, 28, 38, 42, 36, 24, 32]
clust_07 = [35, 13, 31, 34]
clust_08 = [4, 7, 3, 5, 8]
clust_09 = [0]
clust_10 = [1, 2, 9, 6, 10]

#Run the labelling function over each comment
for i in range(len(final_data)):
    label = labelling(final_data["comms"][i])
    final_data["topic_nb"][i] = label[0]
#     final_data["weight"][i] = label[1]
    

#Apply the true label
final_data["topic_label"] = range(len(final_data))

for i in range(len(final_data)):
    if final_data["topic_nb"][i] in clust_01:
        final_data["topic_label"][i] = Topic1

    if final_data["topic_nb"][i] in clust_02:
        final_data["topic_label"][i] = Topic2
        
    if final_data["topic_nb"][i] in clust_03:
        final_data["topic_label"][i] = Topic3
    
    if final_data["topic_nb"][i] in clust_04:
        final_data["topic_label"][i] = Topic4
    
    if final_data["topic_nb"][i] in clust_05:
        final_data["topic_label"][i] = Topic5
    
    if final_data["topic_nb"][i] in clust_06:
        final_data["topic_label"][i] = Topic6
    
    if final_data["topic_nb"][i] in clust_07:
        final_data["topic_label"][i] = Topic7
    
    if final_data["topic_nb"][i] in clust_08:
        final_data["topic_label"][i] = Topic8
    
    if final_data["topic_nb"][i] in clust_09:
        final_data["topic_label"][i] = Topic9
    
    if final_data["topic_nb"][i] in clust_10:
        final_data["topic_label"][i] = Topic10
    
    #Progress bar
    if i == 50290:
        print("25%")

    if i == 100580:
        print("50%")

    if i == 150870:
        print("75%")

    

25%
50%
75%


## Save the results

In [ ]:
final_data.to_csv('Final_data.csv', index=False)

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=5b677b82-ea1e-4dc8-9b2a-3c037ad9d27c' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>